In [27]:
# Data manipulation
import os
import pandas as pd
import csv
import json
import jsonlines as jl
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
from dotenv import load_dotenv
from typing import List
import re

# Machine Learning
import torch
import asyncio
import aiohttp
import torch.nn as nn
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Transformers and Langchain
from pydantic import ValidationError, BaseModel, Field

# API and utility
from huggingface_hub import login
from together import Together
import time
from tqdm import tqdm
import accelerate

load_dotenv('/Users/guida/llm_argument_tasks/.env')

api_key = os.environ.get('TOGETHER_API_KEY')
client = Together(api_key=api_key)

In [28]:
def get_llm(model_type):
    if model_type == "llama":
        return client.chat.completions.create(
            model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
            max_tokens=512,
            messages=[],
            temperature=0.7,
            top_p=0.7,
            top_k=50,
            repetition_penalty=1,
            stop=["<|eot_id|>"]
        )
llm = get_llm("llama")

In [29]:
class ArgumentClassification(BaseModel):
    id: str = Field(description="The ID of the comment being analyzed")
    label: int = Field(description="The label associated with the argument (0 or 1)")

In [30]:
# Dictionary for label-to-argument mappings for each topic
topic_label_to_argument = {
    "abortion": {
        "p-right": "Abortion is a woman’s right.",
        "p-rape": "Rape victims need it to be legal.",
        "p-not_human": "A fetus is not a human yet, so it's okay to abort.",
        "p-mother_danger": "Abortion should be allowed when a mother's life is in danger.",
        "p-baby_ill_treatment": "Unwanted babies are ill-treated by parents and/or not always adopted.",
        "p-birth_ctrl": "Birth control fails at times and abortion is one way to deal with it.",
        "p-not_murder": "Abortion is not murder.",
        "p-sick_mom": "Mother is not healthy/financially solvent.",
        "p-other": "Others",
        "c-adopt": "Put baby up for adoption.",
        "c-kill": "Abortion kills a life.",
        "c-baby_right": "An unborn baby is a human and has the right to live.",
        "c-sex": "Be willing to have the baby if you have sex.",
        "c-bad_4_mom": "Abortion is harmful for women.",
        "c-other": "Others"
    },
    "gayRights": {
        "p-normal": "Gay marriage is like any other marriage.",
        "p-right_denied": "Gay people should have the same rights as straight people.",
        "p-no_threat_for_child": "Gay parents can adopt and ensure a happy life for a baby.",
        "p-born": "People are born gay.",
        "p-religion": "Religion should not be used against gay rights.",
        "p-Other": "Others",
        "c-religion": "Religion does not permit gay marriages.",
        "c-abnormal": "Gay marriages are not normal/against nature.",
        "c-threat_to_child": "Gay parents cannot raise kids properly.",
        "c-gay_problems": "Gay people have problems and create social issues.",
        "c-Other": "Others"
    },
    "obama": {
        "p-economy": "Fixed the economy.",
        "p-War": "Ending the wars.",
        "p-republicans": "Better than the republican candidates.",
        "p-decision_policies": "Makes good decisions/policies.",
        "p-quality": "Has qualities of a good leader.",
        "p-health": "Ensured better healthcare.",
        "p-foreign_policies": "Executed effective foreign policies.",
        "p-job": "Created more jobs.",
        "p-Other": "Others",
        "c-economy": "Destroyed our economy.",
        "c-War": "Wars are still on.",
        "c-job": "Unemployment rate is high.",
        "c-health": "Healthcare bill is a failure.",
        "c-decision_policies": "Poor decision-maker.",
        "c-republicans": "We have better republicans than Obama.",
        "c-quality": "Not eligible as a leader.",
        "c-foreign_policies": "Ineffective foreign policies.",
        "c-Other": "Others"
    },
    "marijuana": {
        "p-not_addictive": "Not addictive.",
        "p-medicine": "Used as a medicine for its positive effects.",
        "p-legal": "Legalized marijuana can be controlled and regulated by the government.",
        "p-right": "Prohibition violates human rights.",
        "p-no_damage": "Does not cause any damage to our bodies.",
        "p-Other": "Others",
        "c-health": "Damages our bodies.",
        "c-mind": "Responsible for brain damage.",
        "c-illegal": "If legalized, people will use marijuana and other drugs more.",
        "c-crime": "Causes crime.",
        "c-addiction": "Highly addictive.",
        "c-Other": "Others"
    }
}

In [31]:
def classify_text(id: str, comment_text: str, topic: str, label: str) -> dict:
    label_to_argument = topic_label_to_argument.get(topic)
    argument = label_to_argument.get(label)

    extract = client.chat.completions.create(
        messages=[
            {"role": "system", "content": f"""
            Analyze the given comment in relation to a specific argument about {topic}. You need to:
            Identify if the comment makes use of the given argument. If it does, assign the label 1. If it does not, assign the label 0.
            Do NOT use any other label.
            Do NOT include the comment or the argument in the response.
            
            The argument to analyze is: {argument}
            
            Provide your response in the following JSON format:
            
            {{
                "id": "{id}",
                "label": "the label for the use of the argument in the comment"
            }}
            
            Analyze the following comment in relation to the given argument:
            """},
            {"role": "user", "content": comment_text},
        ],
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        response_format={
            "type": "json_object",
            "schema": ArgumentClassification.model_json_schema(),
        }
    )
    
    return json.loads(extract.choices[0].message.content)

In [32]:
def process_dataframe_comments(df: pd.DataFrame, topic: str) -> List[dict]:
    results = [] 

    for idx, row in tqdm(df.iterrows(), desc="Processing comments", unit="comment", total=len(df)):
        comment_id = row['id'] 
        comment_text = row['text']  
        argument_text = row['label']
        
        try:
            classification = classify_text(
                id=comment_id, 
                comment_text=comment_text,  
                topic=topic, 
                label=argument_text 
            )

            results.append(classification) 

        except json.JSONDecodeError as e:
            print(f"JSONDecodeError for comment: {comment_text[:50]}... - Error: {e}")
            continue

        except Exception as e:
            print(f"An unexpected error occurred for comment: {comment_text[:50]}... - Error: {e}")
            continue
        
    output_filename = f'yru_{topic}_identification.json'
    with open(output_filename, 'w') as f:
        json.dump(results, f, indent=2)

    return results

In [33]:
ab = pd.read_csv('../../clean_data/yru_abortion.csv')
topic = 'abortion'

process_dataframe_comments(ab, topic)

Processing comments:  80%|███████▉  | 589/739 [36:20<11:10,  4.47s/comment]  

In [22]:
ma = pd.read_csv('../../clean_data/yru_marijuana.csv')
topic = 'marijuana'

process_dataframe_comments(ma, topic)

Processing comments:   1%|          | 4/691 [00:25<1:45:53,  9.25s/comment]

An unexpected error occurred for comment: Marijuana being illegal is part of the reason why ... - Error: Error code: 429 - {"message": "Request was rejected due to request rate limiting. Your rate limits are 60 RPM (1 QPS) and 60000 TPM (1000 TPS). See details: https://docs.together.ai/docs/rate-limits", "type_": "credit_limit"}


Processing comments: 100%|██████████| 691/691 [16:02<00:00,  1.39s/comment]


[{'id': 'ma1', 'label': 1},
 {'id': 'ma1', 'label': 1},
 {'id': 'ma2', 'label': 0},
 {'id': 'ma3', 'label': 1},
 {'id': 'ma3', 'label': 1},
 {'id': 'ma4', 'label': 1},
 {'id': 'ma4', 'label': 1},
 {'id': 'ma4', 'label': 1},
 {'id': 'ma4', 'label': 1},
 {'id': 'ma5', 'label': 0},
 {'id': 'ma6', 'label': 1},
 {'id': 'ma7', 'label': 0},
 {'id': 'ma8', 'label': 1},
 {'id': 'ma8', 'label': 1},
 {'id': 'ma9', 'label': 1},
 {'id': 'ma10', 'label': 1},
 {'id': 'ma11', 'label': 1},
 {'id': 'ma12', 'label': 0},
 {'id': 'ma12', 'label': 0},
 {'id': 'ma13', 'label': 1},
 {'id': 'ma13', 'label': 1},
 {'id': 'ma14', 'label': 0},
 {'id': 'ma15', 'label': 0},
 {'id': 'ma16', 'label': 1},
 {'id': 'ma16', 'label': 1},
 {'id': 'ma16', 'label': 0},
 {'id': 'ma17', 'label': 0},
 {'id': 'ma17', 'label': 0},
 {'id': 'ma18', 'label': 0},
 {'id': 'ma19', 'label': 1},
 {'id': 'ma20', 'label': 0},
 {'id': 'ma21', 'label': 1},
 {'id': 'ma22', 'label': 1},
 {'id': 'ma22', 'label': 1},
 {'id': 'ma23', 'label': 0},


In [23]:
oba = pd.read_csv('../../clean_data/yru_obama.csv')
topic = 'obama'

process_dataframe_comments(oba, topic)

Processing comments:   0%|          | 0/646 [00:00<?, ?comment/s]

Processing comments: 100%|██████████| 646/646 [14:31<00:00,  1.35s/comment]


[{'id': 'oba1', 'label': 1},
 {'id': 'oba2', 'label': 0},
 {'id': 'oba3', 'label': 1},
 {'id': 'oba4', 'label': 0},
 {'id': 'oba5', 'label': 1},
 {'id': 'oba6', 'label': 0},
 {'id': 'oba7', 'label': 1},
 {'id': 'oba7', 'label': 1},
 {'id': 'oba8', 'label': 0},
 {'id': 'oba8', 'label': 1},
 {'id': 'oba9', 'label': 1},
 {'id': 'oba9', 'label': 1},
 {'id': 'oba10', 'label': 1},
 {'id': 'oba10', 'label': 1},
 {'id': 'oba11', 'label': 0},
 {'id': 'oba12', 'label': 1},
 {'id': 'oba13', 'label': 1},
 {'id': 'oba14', 'label': 0},
 {'id': 'oba15', 'label': 1},
 {'id': 'oba16', 'label': 0},
 {'id': 'oba16', 'label': 0},
 {'id': 'oba17', 'label': 1},
 {'id': 'oba17', 'label': 1},
 {'id': 'oba17', 'label': 1},
 {'id': 'oba18', 'label': 1},
 {'id': 'oba18', 'label': 1},
 {'id': 'oba19', 'label': 1},
 {'id': 'oba20', 'label': 1},
 {'id': 'oba20', 'label': 1},
 {'id': 'oba21', 'label': 1},
 {'id': 'oba22', 'label': 0},
 {'id': 'oba23', 'label': 1},
 {'id': 'oba24', 'label': 1},
 {'id': 'oba25', 'labe

In [26]:
gm = pd.read_csv('../../clean_data/yru_gayrights.csv')
topic = 'gayRights'

process_dataframe_comments(gm, topic)

Processing comments:   0%|          | 0/772 [00:00<?, ?comment/s]

Processing comments: 100%|██████████| 772/772 [17:20<00:00,  1.35s/comment]


[{'id': 'gr1', 'label': 1},
 {'id': 'gr1', 'label': 1},
 {'id': 'gr1', 'label': 1},
 {'id': 'gr2', 'label': 1},
 {'id': 'gr3', 'label': 1},
 {'id': 'gr4', 'label': 0},
 {'id': 'gr5', 'label': 1},
 {'id': 'gr6', 'label': 1},
 {'id': 'gr7', 'label': 1},
 {'id': 'gr8', 'label': 1},
 {'id': 'gr9', 'label': 1},
 {'id': 'gr9', 'label': 1},
 {'id': 'gr10', 'label': 0},
 {'id': 'gr11', 'label': 0},
 {'id': 'gr12', 'label': 0},
 {'id': 'gr13', 'label': 1},
 {'id': 'gr13', 'label': 1},
 {'id': 'gr13', 'label': 1},
 {'id': 'gr14', 'label': 1},
 {'id': 'gr15', 'label': 1},
 {'id': 'gr15', 'label': 1},
 {'id': 'gr16', 'label': 1},
 {'id': 'gr16', 'label': 1},
 {'id': 'gr17', 'label': 0},
 {'id': 'gr17', 'label': 1},
 {'id': 'gr18', 'label': 1},
 {'id': 'gr19', 'label': 1},
 {'id': 'gr20', 'label': 1},
 {'id': 'gr20', 'label': 0},
 {'id': 'gr21', 'label': 1},
 {'id': 'gr22', 'label': 1},
 {'id': 'gr22', 'label': 1},
 {'id': 'gr23', 'label': 0},
 {'id': 'gr24', 'label': 1},
 {'id': 'gr25', 'label': 0